In [61]:
dni = 77432071
p = next_prime(7743)
q = next_prime(2071)

p, q

(7753, 2081)

In [62]:
n = p*q

In [63]:
e = 11
p = 11
primes = Primes()

while p.gcd(euler_phi(n)) != 1:
    p = primes.next(p)

e = p
e

11

In [64]:
d = inverse_mod(e, euler_phi(n))
d

5863331

¿Lo hemos calculado bien?

In [65]:
(e * d) % euler_phi(n)

1

## Apartado 1

In [66]:
m = 0xCAFE

In [67]:
def cifrar(m, e, n):
    return power_mod(m, e, n)

In [68]:
c = cifrar(m, e, n)
c

10576233

En hexa:

In [69]:
'0x' + c.hex()

'0xa16169'

## Apartado 2

In [70]:
def descifrar(c, e, n):
    d = inverse_mod(e, euler_phi(n))
    return power_mod(c, d, n)

In [71]:
m = descifrar(c, e, n)
'0x' + m.hex()

'0xcafe'

## Apartado 3

In [72]:
b_valores = list(range(2, 9))
b_valores

[2, 3, 4, 5, 6, 7, 8]

## Apartado 4 

Tomamos el polinomio $(x - p)(x - q) = x^2 - (p + q)x + n$. Puesto que $\varphi(n) = n + 1 - (p + q)$, se tiene que 

$$
(x - p)(x - q) = x^2 - (n + 1 - \varphi(n))x - n
$$

Claramente, las raíces del polinomio son $p$ y $q$, por lo que podemos hallar sus valores resolviendo la ecuación 

$$
\begin{cases}
p, q = b \pm \sqrt{b^2 - n} \\
p + q = 2b \\
p + q = n - \varphi(n) + 1 
\end{cases}
$$

Así que 

$$
b = \frac{n - \varphi(n) + 1 }{2}
$$

In [73]:
b = (n - euler_phi(n) + 1)/2
b

4917

In [74]:
p = b + sqrt(b^2 - n)
q = b - sqrt(b^2 - n)
p, q

(7753, 2081)

## Apartado 5

Queremos factorizar $n$ a partir de $e$ y $d$ esta vez.

In [75]:
k = e*d - 1
k

64496640

Consideramos los primeros primos hasta 19

In [78]:
for a in [2, 3, 5, 7, 11, 13, 17, 19]:
    print(power_mod(a, k, n))

1
1
1
1
1
1
1
1


In [100]:
def factorizar_n(e, d, n):
    k = e*d - 1
    
    debemos_parar = False
    
    while not debemos_parar:
        for a in [2, 3, 5, 7, 11, 13, 17, 19]:
            actual = power_mod(a, k, n)
            print(str(a) + "^" + str(k) + " mod n = " + str(actual))
            if actual != 1:
                debemos_parar = True
                solucion = actual
        
        k = k // 2
        print("\n")
    
    return [n.gcd(solucion - 1), n.gcd(solucion + 1)]

In [101]:
factorizar_n(e, d, n)

2^64496640 mod n = 1
3^64496640 mod n = 1
5^64496640 mod n = 1
7^64496640 mod n = 1
11^64496640 mod n = 1
13^64496640 mod n = 1
17^64496640 mod n = 1
19^64496640 mod n = 1


2^32248320 mod n = 1
3^32248320 mod n = 1
5^32248320 mod n = 1
7^32248320 mod n = 1
11^32248320 mod n = 1
13^32248320 mod n = 1
17^32248320 mod n = 1
19^32248320 mod n = 1


2^16124160 mod n = 1
3^16124160 mod n = 1
5^16124160 mod n = 1
7^16124160 mod n = 1
11^16124160 mod n = 1
13^16124160 mod n = 1
17^16124160 mod n = 1
19^16124160 mod n = 1


2^8062080 mod n = 1
3^8062080 mod n = 1
5^8062080 mod n = 1
7^8062080 mod n = 1
11^8062080 mod n = 1
13^8062080 mod n = 1
17^8062080 mod n = 1
19^8062080 mod n = 1


2^4031040 mod n = 1
3^4031040 mod n = 1
5^4031040 mod n = 1
7^4031040 mod n = 1
11^4031040 mod n = 1
13^4031040 mod n = 1
17^4031040 mod n = 1
19^4031040 mod n = 1


2^2015520 mod n = 1
3^2015520 mod n = 1
5^2015520 mod n = 1
7^2015520 mod n = 1
11^2015520 mod n = 1
13^2015520 mod n = 1
17^2015520 mod n = 1
19^

[7753, 2081]

Así que hemos conseguido factorizar $n$